In [14]:
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
#create dicrionnary with 
emission_factors = {'Strom-Mix-D': 0.137808, 'Fotovoltaik': 0.032308}
paths ={'NT-WN': r"C:\Users\lbarnic\OneDrive - Fraunhofer\A_Bearbeitung\9_Parameterstudie_REP\9_2_VLT_Variation\9_VL_RL_optimistisch\Simulationen\II_NT_WN\Co2e_NT_WN.out",
        'KNW': r"C:\Users\lbarnic\OneDrive - Fraunhofer\A_Bearbeitung\9_Parameterstudie_REP\9_2_VLT_Variation\9_VL_RL_optimistisch\Simulationen\III_kaltes_Nahwaermenetz\Co2e_kalte_NW.out",
        'dezentral': r"C:\Users\lbarnic\OneDrive - Fraunhofer\A_Bearbeitung\9_Parameterstudie_REP\9_2_VLT_Variation\9_VL_RL_optimistisch\Simulationen\I_dez\Co2e_dezentral.out",
        }
         
def import_data(path):
    # system = pd.read_csv(path)
    system = pd.read_csv(path, sep=',', skipinitialspace=True, index_col=0, skiprows=7)
    # remove spaces in column names
    system.columns = system.columns.str.strip()
    #drop last column
    system = system.iloc[:, :-1]
    system["PV-Eigennutzung"] = pd.NA
    return system

result = {}
systems = {}

for key in paths:
    system = import_data(paths[key]).sum()/1000/4 #MWh
    systems[key] = system
    Netzstrom_co2e=system["Netzstrom"]*emission_factors["Strom-Mix-D"]*25 # tCO2e
    PV_co2e=system["PV_Strom"]*emission_factors["Fotovoltaik"]*25
    co2e_system = Netzstrom_co2e+PV_co2e
    result[key] = co2e_system
#aufsummieren je spalte
#multipliziere Wert Netzstrom mit Strommix
pd.Series(result)

NT-WN        341.067346
KNW          148.167136
dezentral    172.460817
dtype: float64

In [16]:
# Dataframe aller Systeme in MWh in kWh
df = pd.DataFrame(systems)*1000
df

,NT-WN,KNW,dezentral
Strom_system,113467.012445,46297.879741,46297.879741
Netzstrom,87028.011217,31037.006321,38088.463712
PV_Strom,51056.63237,51056.63237,51056.63237
Netzeinspeisung,43168.825386,47656.686468,49997.706776
PV-Eigennutzung,0.0,0.0,0.0


# Sondendimensionerung


In [17]:
## import der Daten zur Sondendimensioerung

import numpy as np
# BASIS
#input data from path als csv datei als df buildings_dfs_basis
path = r"C:\Users\lbarnic\OneDrive - Fraunhofer\A_Bearbeitung\5_Ermittlung_der_Sanierungspotenziale_REP\5_4_Bestimmung_Lastprofile\Input_Data\Basissanierung\buildings_dfs_basis.csv"
buildings_dfs_basis = pd.read_csv(path, index_col=0, header=[0,1])
buildings_dfs_basis = buildings_dfs_basis.drop(columns=[col for col in buildings_dfs_basis.columns if 'QCOOL' in col])

# audsummieren je Zeile
buildings_dfs_basis_sum = buildings_dfs_basis.sum(axis=1)
buildings_dfs_basis_sum = buildings_dfs_basis_sum.sort_values(ascending=False).reset_index(drop=True)

# AMBITIONIERT
import numpy as np
#input data from path als csv datei als df buildings_dfs_basis
path = r"C:\Users\lbarnic\OneDrive - Fraunhofer\A_Bearbeitung\5_Ermittlung_der_Sanierungspotenziale_REP\5_4_Bestimmung_Lastprofile\Input_Data\Ambitionierte_Sanierung\buildings_dfs_ambitioniert.csv"
buildings_dfs_ambitioniert = pd.read_csv(path, index_col=0, header=[0,1])
buildings_dfs_ambitioniert = buildings_dfs_ambitioniert.drop(columns=[col for col in buildings_dfs_ambitioniert.columns if 'QCOOL' in col])


In [18]:
#drop multiindex
buildings_dfs_ambitioniert = buildings_dfs_ambitioniert.droplevel(1,axis=1)
buildings_dfs_ambitioniert

,FVR,SB,SpS,TV
0,0.0,0.000000,0.000000,0.000000
1,0.0,0.000000,12.539251,0.047962
2,0.0,0.000000,12.000154,0.000000
3,0.0,0.000000,12.239547,0.000000
4,0.0,0.000000,11.914561,0.090431
...,...,...,...,...
35036,0.0,1.656730,12.849238,1.041520
35037,0.0,1.677870,13.150169,1.065593
35038,0.0,1.677841,13.453540,1.092630
35039,0.0,1.677895,13.795780,1.123399


In [19]:
COP_Ambitioniert_o = {
    'FVR': 5.18,
    'SB': 5.18,
    'SpS': 5.18,
    'TV': 5.18}

COP_basis_o = 4.62

# multipliziere COP des Gebäude mit Last des Gebäudes
buildings_dfs_ambitioniert_Umweltwaerme = buildings_dfs_ambitioniert * (1-1/pd.Series(COP_Ambitioniert_o))
Verdampferleistung_KNW = buildings_dfs_ambitioniert_Umweltwaerme.sum(axis=1).quantile(0.975) # je Zeitschritt 97,5% Quantil

#dezentral
Verdampferleistung_dez = buildings_dfs_ambitioniert_Umweltwaerme.quantile(0.975).sum(axis=0) # je Zeitschritt 97,5% Quantil
print(Verdampferleistung_dez)
print(Verdampferleistung_KNW)

#NTWN
Verdampferleistung_NTWN = buildings_dfs_basis_sum.quantile(0.975) * (1-1/COP_basis_o)
print(Verdampferleistung_NTWN)


111.56342575201558
106.72798427306832
150.54692928530721
